# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [108]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

from panel.interact import interact
from panel import widgets

pn.extension()

In [109]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

# Import Data

In [110]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [111]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""    
    # YOUR CODE HERE!
    sfo_housing_unit = sfo_data["housing_units"].groupby("year").sum()
    housing_unit_plot = sfo_housing_unit.hvplot.bar(
                title = "Total Housing Units per Year",
                x="year",
                y="housing_units",
                ylim = (sfo_housing_unit.min()-sfo_housing_unit.std(), sfo_housing_unit.max()+sfo_housing_unit.std())
        )
    return housing_unit_plot


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    # YOUR CODE HERE!
    sfo_avg_gross_rent = sfo_data["gross_rent"].groupby("year").mean()
    avg_gross_rent_plot = sfo_avg_gross_rent.hvplot.line(
        title="Average Gross Rent per Year",
        x="year",
        y="gross_rent"
        )
    return avg_gross_rent_plot



def average_sales_price():
    """Average Sales Price Per Year."""   
    # YOUR CODE HERE!
    sfo_avg_sales_price = sfo_data["sale_price_sqr_foot"].groupby("year").mean()
    avg_sales_price_plot = sfo_avg_sales_price.hvplot.line(
        title="Average Sales Price per Year",
        x="year",
        y="sale_price_sqr_foot"
        )
    return avg_sales_price_plot

def average_price_by_neighborhood(neighborhood):
    """Average Prices by Neighborhood.""" 
    # YOUR CODE HERE!    
    neighbothood_plot = sfo_housing_neighborhood[sfo_housing_neighborhood["neighborhood"]==neighborhood].hvplot.line(
        x="year",
        y="sale_price_sqr_foot",
        title=f"Average Price per Sqft in {neighborhood}",
    )   
    return neighbothood_plot
    

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""   
    # YOUR CODE HERE!
    sfo_neighborhood_top10 = sfo_data.groupby("neighborhood").mean().sort_values("sale_price_sqr_foot", ascending=False).head(10).reset_index()
    neighborhood_top10_plot = sfo_neighborhood_top10.hvplot.bar(
    title = "Top 10 Expensive Neighborhood in SFO",
    x = "neighborhood",
    xlabel = "Neighborhood",
    y = "sale_price_sqr_foot",
    ylabel = "Avg. Price per Sqft.",
    rot = 90   
)    
    return neighborhood_top10_plot


def parallel_coordinates():
    """Parallel Coordinates Plot."""   
    # YOUR CODE HERE!
    sfo_neighborhood_top10 = sfo_data.groupby("neighborhood").mean().sort_values("sale_price_sqr_foot", ascending=False).head(10).reset_index()
    parallel_coor_plot = px.parallel_coordinates(sfo_neighborhood_top10,  color="sale_price_sqr_foot")
    return parallel_coor_plot


def parallel_categories():
    """Parallel Categories Plot."""   
    # YOUR CODE HERE!
    sfo_neighborhood_top10 = sfo_data.groupby("neighborhood").mean().sort_values("sale_price_sqr_foot", ascending=False).head(10).reset_index()
    parallel_cate_plot = px.parallel_categories(
    sfo_neighborhood_top10,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units","gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhood",
        "sale_price_sqr_foot": "Avg. Price per Sqft.",
        "housing_units": "Housing Units",
        "gross_rent":"Gross Rental"
    },
)
    return parallel_cate_plot


def neighborhood_map():
    """Neighborhood Map"""    
    # YOUR CODE HERE!
    sfo_map = sfo_data.groupby("neighborhood").mean().reset_index()
    sfo_concat = pd.concat([df_neighborhood_locations,sfo_map], axis="columns", join="inner").drop(columns=["neighborhood"])
    load_dotenv()
    map_box_api = os.getenv("MAPBOX")
    px.set_mapbox_access_token(map_box_api)
    map_plot = px.scatter_mapbox(
    sfo_concat,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    zoom=10)
    return map_plot
# Display the map
    map_plot.show()


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [112]:
# YOUR CODE HERE!
annual_analysis = pn.Column(
    "##Annual Analysis", housing_units_per_year(), average_gross_rent(), average_sales_price()
)

sfo_housing_neighborhood = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()
list_of_neighborhood = sfo_housing_neighborhood["neighborhood"].unique()
neighborhood_analysis = pn.Column(
    "##Neighborhood Analysis", interact(average_price_by_neighborhood, neighborhood=list_of_neighborhood), top_most_expensive_neighborhoods()
)

parallel_analysis = pn.Column(
    "Parallel Analysis", parallel_coordinates(), parallel_categories()
)

panel = pn.Tabs(
    ("Annual", annual_analysis), ("Neighborhood", neighborhood_analysis), ("Parallel", parallel_analysis), ("Map",neighborhood_map())
)

## Serve the Panel Dashboard

In [113]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive12893')
        [2] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Plotly(Figure, name='Map')